# Tensorboard

---
---

**NOTE: You must watch the corresponding video to understand this lecture. This notebook can't serve as a full guide. Please watch the video BEFORE posting questions to the QA forum.**

---
---

Let's explore the built in data visualization capabilities that come with Tensorboard.

Full official tutorial available here: https://www.tensorflow.org/tensorboard/get_started

## Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('cancer_classification.csv')

### Train Test Split

In [3]:
X = df.drop('benign_0__mal_1',axis=1).values
y = df['benign_0__mal_1'].values

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=101)


### Scaling Data

In [6]:
from sklearn.preprocessing import MinMaxScaler

In [7]:
scaler = MinMaxScaler()

In [8]:
scaler.fit(X_train)

MinMaxScaler()

In [9]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Creating the Model

In [10]:
import tensorflow as tf
k = tf.keras

2023-04-29 15:50:26.048007: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-29 15:50:26.098624: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-29 15:50:26.099143: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 15:50:26.968346: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
early_stop = k.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

## Creating the Tensorboard Callback

TensorBoard is a visualization tool provided with TensorFlow.

This callback logs events for TensorBoard, including:
* Metrics summary plots
* Training graph visualization
* Activation histograms
* Sampled profiling

If you have installed TensorFlow with pip, you should be able
to launch TensorBoard from the command line:

```sh
tensorboard --logdir=path_to_your_logs
```

You can find more information about TensorBoard
[here](https://www.tensorflow.org/tensorboard/).

    Arguments:
        log_dir: the path of the directory where to save the log files to be
          parsed by TensorBoard.
        histogram_freq: frequency (in epochs) at which to compute activation and
          weight histograms for the layers of the model. If set to 0, histograms
          won't be computed. Validation data (or split) must be specified for
          histogram visualizations.
        write_graph: whether to visualize the graph in TensorBoard. The log file
          can become quite large when write_graph is set to True.
        write_images: whether to write model weights to visualize as image in
          TensorBoard.
        update_freq: `'batch'` or `'epoch'` or integer. When using `'batch'`,
          writes the losses and metrics to TensorBoard after each batch. The same
          applies for `'epoch'`. If using an integer, let's say `1000`, the
          callback will write the metrics and losses to TensorBoard every 1000
          samples. Note that writing too frequently to TensorBoard can slow down
          your training.
        profile_batch: Profile the batch to sample compute characteristics. By
          default, it will profile the second batch. Set profile_batch=0 to
          disable profiling. Must run in TensorFlow eager mode.
        embeddings_freq: frequency (in epochs) at which embedding layers will
          be visualized. If set to 0, embeddings won't be visualized.
       

In [12]:
from datetime import datetime

In [13]:
datetime.now().strftime("%Y-%m-%d--%H%M")

'2023-04-29--1550'

In [14]:
# WINDOWS: Use "logs\\fit"
# MACOS/LINUX: Use "logs\fit"

log_directory = 'logs/fit'

# OPTIONAL: ADD A TIMESTAMP FOR UNIQUE FOLDER
# timestamp = datetime.now().strftime("%Y-%m-%d--%H%M")
# log_directory = log_directory + '\\' + timestamp


board = k.callbacks.TensorBoard(log_dir=log_directory,histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq='epoch',
    profile_batch=2,
    embeddings_freq=1)

2023-04-29 15:50:27.953445: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2023-04-29 15:50:27.953495: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2023-04-29 15:50:27.975594: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


Now create the model layers:

In [15]:
model = k.Sequential()
model.add(k.layers.Dense(units=30,activation='relu'))
model.add(k.layers.Dropout(0.5))
model.add(k.layers.Dense(units=15,activation='relu'))
model.add(k.layers.Dropout(0.5))
model.add(k.layers.Dense(units=1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

2023-04-29 15:50:28.064820: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


## Train the Model

In [16]:
model.fit(x=X_train, 
          y=y_train, 
          epochs=600,
          validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stop,board]
          )

Epoch 1/600
 1/14 [=>............................] - ETA: 12s - loss: 0.6918

2023-04-29 15:50:29.278332: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2023-04-29 15:50:29.278365: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2023-04-29 15:50:29.282561: I tensorflow/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2023-04-29 15:50:29.283939: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


14/14 [==============================] - 2s 52ms/step - loss: 0.6843 - val_loss: 0.6689
Epoch 2/600
14/14 [==============================] - 0s 17ms/step - loss: 0.6731 - val_loss: 0.6538
Epoch 3/600
14/14 [==============================] - 0s 19ms/step - loss: 0.6641 - val_loss: 0.6414
Epoch 4/600
14/14 [==============================] - 0s 12ms/step - loss: 0.6576 - val_loss: 0.6283
Epoch 5/600
14/14 [==============================] - 0s 10ms/step - loss: 0.6354 - val_loss: 0.6134
Epoch 6/600
14/14 [==============================] - 0s 10ms/step - loss: 0.6166 - val_loss: 0.5934
Epoch 7/600
14/14 [==============================] - 0s 12ms/step - loss: 0.5925 - val_loss: 0.5658
Epoch 8/600
14/14 [==============================] - 0s 20ms/step - loss: 0.5755 - val_loss: 0.5371
Epoch 9/600
14/14 [==============================] - 0s 14ms/step - loss: 0.5452 - val_loss: 0.5052
Epoch 10/600
14/14 [==============================] - 0s 9ms/step - loss: 0.5329 - val_loss: 0.4746
Epoch 11/600

# Running Tensorboard


## Running through the Command Line

**Watch video to see how to run Tensorboard through a command line call.**

Tensorboard will run locally in your browser at [http://localhost:6006/](http://localhost:6006/)


In [17]:
print(log_directory)

logs/fit


### Use cd at your command line to change directory to the file path reported back by pwd or your current .py file location.
### Then run this code at your command line or terminal

In [18]:
# tensorboard --logdir logs/fit